In [1]:
pip install deepface

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import urllib.request
import shutil
import os
import bz2
import threading

# Download the shape predictor file
url = "https://github.com/davisking/dlib-models/raw/master/shape_predictor_68_face_landmarks.dat.bz2"
filename_bz2 = "shape_predictor_68_face_landmarks.dat.bz2"
filename_dat = r"C:\Users\COMPUMARTS\final_facial\shape_predictor_68_face_landmarks.dat\shape_predictor_68_face_landmarks.dat"

# Download the file
with urllib.request.urlopen(url) as response, open(filename_bz2, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

# Decompress the file
with open(filename_bz2, 'rb') as f_in, open(filename_dat, 'wb') as f_out:
    decompressed_data = bz2.decompress(f_in.read())
    f_out.write(decompressed_data)


In [2]:
import urllib.request
import shutil
import os
import bz2
import cv2
import dlib
from deepface import DeepFace
import socket

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(filename_dat)

In [4]:
def facial():
    global data
    cap = cv2.VideoCapture(0)
    frame_count = 0

    # Set initial emotion text
    emotion_text = "Emotion: Unknown"
    while cap.isOpened():
        ret, frame = cap.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = detector(gray)
        cpy = frame.copy()

        for face in faces:
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()

            cv2.rectangle(cpy, (x1, y1), (x2, y2), (0, 255, 0), 5)

            shape = predictor(gray, face)

            for n in range(0, 68):
                x = shape.part(n).x
                y = shape.part(n).y
                cv2.circle(cpy, (x, y), 1, (0, 0, 255), 3)
        if frame_count % 60 == 0:  
            try:

                required_outputs =  ['emotion']
                result = DeepFace.analyze(cpy,actions = required_outputs )
                emotion_text = f"Emotion: { result[0]['dominant_emotion']}"
                data=result[0]['dominant_emotion']
            except Exception as e:
                print("Error analyzing emotion:", str(e))

        # Display emotion text in the top-left corner
        cv2.putText(cpy, emotion_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        frame_count += 1


        cv2.imshow('Face Landmarks', cpy)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    


In [ ]:
def handle_client(conn, addr):
    print(f"Accepted connection from {addr}")

    # Perform any actions you want with the connection here
    # For example, you can call your facial recognition function
    facial()

    while True:
        try:
            # Receive data from the client
            data = conn.recv(1024)
            if not data:
                break

            # Process the received data (if needed)
            # ...

            # Send a response back to the client
            msg = bytes(data, 'utf-8')
            conn.send(msg)
        except Exception as e:
            print(f"Error handling connection from {addr}: {e}")
            break

    # Close the connection when done
    conn.close()
    print(f"Connection from {addr} closed")

def start_server():
    host = 'localhost'
    port = 5200

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(5)

    print(f"Server listening on {host}:{port}")

    while True:
        try:
            conn, addr = server_socket.accept()
            client_thread = threading.Thread(target=handle_client, args=(conn, addr))
            client_thread.start()
        except Exception as e:
            print(f"Error accepting connection: {e}")

if __name__ == "__main__":
    start_server()

Server listening on localhost:5200
Accepted connection from ('127.0.0.1', 59984)


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.22it/s]


Error analyzing emotion: Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.01it/s]


Connection from ('127.0.0.1', 59984) closed
Accepted connection from ('127.0.0.1', 52209)
Error analyzing emotion: Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error analyzing emotion: Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.69it/s]


Error analyzing emotion: Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.73it/s]


Error analyzing emotion: Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error handling connection from ('127.0.0.1', 52209): [WinError 10054] An existing connection was forcibly closed by the remote host
Connection from ('127.0.0.1', 52209) closed


In [ ]:
# MySocket = socket.socket()
# MySocket.bind(('localhost', 5500))
# MySocket.listen(5)
# conn, addr = MySocket.accept()
# print("Device connected")

# msg = bytes(data, 'utf-8')
# conn.send(msg)
    

In [ ]:
# MySocket = socket.socket()
# MySocket.bind(('localhost', 5200))
# MySocket.listen(5)
# conn, addr = MySocket.accept()
# print("Device connected")
# facial()
# while True:
#     msg = bytes(data, 'utf-8')
#     conn.send(msg)
#     break